In [1]:
from QuantileRegression import *
import numpy as np
from sympy import lambdify, symbols, chebyshevt

import plotly.express as px
import plotly.graph_objects as go

-------

## Generate input data

In [2]:
np.random.seed(0)
x = np.linspace(0, 2, 240)
y = np.sin(2 * np.pi * x) + np.random.normal(0, 0.4, x.shape)
data = np.column_stack((x, y))

Plot the generated data:

In [11]:
fig = px.scatter(x=data[:, 0], y=data[:, 1], labels={'x': 'X-axis', 'y': 'Y-axis'}, template='plotly_dark')
fig.show()

------

## Fit given functions

Define a list of functions:

In [4]:
funcs = [lambda x: 1, lambda x: x, lambda x: np.cos(x), lambda x: np.cos(3 * x), lambda x: np.cos(6 * x)]

In [5]:
y = symbols('y')
chebyshev_polynomials = [chebyshevt(n, y) for n in range(10)]
chebyshev_polynomials

[1,
 y,
 2*y**2 - 1,
 4*y**3 - 3*y,
 8*y**4 - 8*y**2 + 1,
 16*y**5 - 20*y**3 + 5*y,
 32*y**6 - 48*y**4 + 18*y**2 - 1,
 64*y**7 - 112*y**5 + 56*y**3 - 7*y,
 128*y**8 - 256*y**6 + 160*y**4 - 32*y**2 + 1,
 256*y**9 - 576*y**7 + 432*y**5 - 120*y**3 + 9*y]

In [6]:
def chebyshev_t_polynomials(n):
    if n == 0:
        return lambda x: 1
    elif n == 1:
        return lambda x: x
    else:
        T0 = lambda x: 1
        T1 = lambda x: x
        for i in range(2, n + 1):
            Tn = lambda x, T0=T0, T1=T1: 2 * x * T1(x) - T0(x)
            T0, T1 = T1, Tn
        return Tn

chebyshev_polynomials = [chebyshev_t_polynomials(i) for i in range(10)]

Define ***regression quantile*** probabilities:

In [7]:
probs = [0.1, 0.5, 0.9]

Perform quantile regression:

In [8]:
qr_results = quantile_regression_fit(data, chebyshev_polynomials, probs)

Plot the obtained regression quantilies:

In [10]:
# Create a plotly figure
fig = go.Figure()

fig.add_trace(go.Scatter(x=data[:, 0], y=data[:, 1], mode="markers", name="data"))

fig.update_layout(
    xaxis_title='X Axis Label',
    yaxis_title='Y Axis Label',
    template='plotly_dark'
)

# Plot each regression quantile
for i, q in enumerate(probs):
    y_fit = [qr_results[i](xi) for xi in x]
    fig.add_trace(go.Scatter(x=x, y=y_fit, mode='lines', name=f'{q}'))
    
fig.show()